# Filtering

### LIMIT

- In many (testing) settings, we do not actually want our query to print all relevant lines <br>
- Say, for example, we want to check that our query returns all BLX assets
- Printing only the first e.g. 5 results may suffice to check that the query has the desired behaviour

In [ ]:
SELECT *
FROM customers
LIMIT 5;

### ORDER BY (recap)

- We can order records however we like, by a single or by multiple columns
- Default sorting order in Postgres is ascending (ASC). For legibility of code, nice to add manually
- Use DESC to enforce descending sorting order

In [ ]:
SELECT *
FROM customers
ORDER BY age ASC, last_name DESC
LIMIT 20;

### WHERE (recap)

In [ ]:
SELECT *
FROM customers
WHERE first_name = 'Brian';

# Basic mathematics and logic

### Arithmetic
- In lesson 1 we already saw an application of addition and multiplication

In [ ]:
UPDATE customers
SET id = 2*id + 15;

- Subtraction and division follow logically. In a very simple query:

In [ ]:
SELECT 6-2;
SELECT 6/2;

- One thing to be very careful of is <i>integer division</i>
- Postgres handles division of integers as some programming languages do: return the integer quotient and ignore the remainder

In [ ]:
SELECT 3/2;

- To avoid this behaviour, we have to carefully choose types
- If Postgres knows that we are dealing with decimals, then it will treat the numbers as such during e.g. division

In [ ]:
SELECT 3.0/2

### A quick note on floating point numbers

- As above, you may find that Postgres sometimes returns an odd number when you do arithmetic (e.g. numbers ending in .000000000000 or .9999999999999)
- This has to do with how computers store numbers
- In some programming languages, whether it happens actually depends on the variable type
- Rounding can be a useful tool to avoid ending up with such numbers

In [ ]:
SELECT ROUND(3.0/2, 2);

### Other mathematical functions and operators
- Postgres contains many other mathematical functions and operators (modulus, exponents, absolute value, ...)
- In general, I would prefer using a function to an operator for uncommon operators, abs(-3) is easier to understand than @-3
- See https://www.postgresql.org/docs/12/functions-math.html for a full overview

### (In)equalities/comparisons

The usual inequality operators are all available

In [ ]:
SELECT *
FROM customers
WHERE age >= 18
LIMIT 5;

SELECT *
FROM customers
WHERE age <> 18
LIMIT 5;

Remember that equality testing uses a single =, == will lead to errors

In [ ]:
SELECT *
FROM customers
WHERE age = 18
LIMIT 5;

### Testing NULL
- A special case involves testing if a result 'equals' or does not 'equal' NULL <br>
- <strong>Always use IS NULL or IS NOT NULL</strong> for this, do not write this test manually (e.g. <> NULL) <br>

In [ ]:
INSERT INTO customers (first_name, last_name)
VALUES ('first', 'last');

SELECT *
FROM customers
WHERE first_name = 'first';

-- Do not do this!
SELECT *
FROM customers
WHERE age = NULL;

- In general, Postgres will tell you that NULL = NULL is false!

### Logic: AND, OR, NOT

In [ ]:
SELECT *
FROM customers
WHERE   age >= 18
    AND first_name = 'Brian';

In [ ]:
SELECT *
FROM customers
WHERE   age >= 18
    OR first_name = 'Brian';

A condition can be negated explicitly by using NOT

In [ ]:
SELECT *
FROM customers
WHERE   age >= 18
    AND NOT(first_name = 'Brian');

### Using ranges: BETWEEN

BETWEEN: a convenient way of making queries involving ranges more legible

In [ ]:
SELECT *
FROM customers
WHERE   age >= 18
    AND age <= 30;
    
SELECT *
FROM customers
WHERE age BETWEEN 18 AND 30;

In [ ]:
SELECT *
FROM customers
WHERE  age < 18
    OR age > 30;
    
SELECT *
FROM customers
WHERE age NOT BETWEEN 18 AND 30;

# Basic pattern matching and sets

### Pattern matching: LIKE, ILIKE and wildcards

 - LIKE allows us to filter queries based on <i>similar</i> strings

In [ ]:
SELECT *
FROM customers
WHERE first_name LIKE 'Brian';

- ILIKE is a Postgres-specific extension that also allows for case-insensitive matching

In [ ]:
SELECT *
FROM customers
WHERE first_name LIKE 'brian';

SELECT *
FROM customers
WHERE first_name ILIKE 'brian';

- The power of pattern matching is in the use of 'wildcards'; without them, LIKE is not different from =
- Wildcards are 'placeholders' that allow any value, making it very easy to match strings that contain <i>something</i>
- Postgres has two wildcards: underscore _ is a placeholder for a single character, percentage sign % is a placeholder for any number of characters

In [ ]:
SELECT *
FROM customers
WHERE first_name LIKE 'Bria_';

SELECT *
FROM customers
WHERE first_name LIKE 'Bri__';

SELECT *
FROM customers
WHERE first_name LIKE 'Bri%';

SELECT *
FROM customers
WHERE first_name LIKE '%ri%';

- Wildcards allow us to make some fairly elaborate filters
- Find any email address with a 6-character account name, ending in .org:

In [ ]:
SELECT *
FROM customers
WHERE email LIKE '______@%.org'

- As before, we can use NOT to explicitly avoid returning a pattern

In [ ]:
SELECT *
FROM customers
WHERE first_name NOT LIKE 'Bri%'
LIMIT 5;

### Escape characters
- How do we find strings that actually a percentage sign?
- For this we use the backward slash \ as 'escape character'
- The escape character tells Postgres that we want it to interpret the % literally

In [ ]:
INSERT INTO customers (first_name) VALUES ('aa%bb');

SELECT *
FROM customers
WHERE first_name LIKE '%\%%';

Question: how do we now go about finding strings that contain a '\\'?

### Using sets: IN and ANY
- The OR keyword we have seen thus far can become quite cumbersome to use
- IN is a more convenient and legible way of allowing multiple values

In [ ]:
SELECT *
FROM customers
WHERE first_name = 'Tyler' OR first_name = 'Claudia';

SELECT *
FROM customers
WHERE first_name IN ('Tyler', 'Claudia', 'Brian')

(Slightly more advanced)
- As an alternative for IN we can use ANY
- ANY is more flexible than IN: it allows for more than just equalities
- This can be very useful if combined with LIKE
- Unlike IN, we must pass an array to ANY, because ANY takes a single column as input

SELECT *
FROM customers
WHERE first_name = ANY (ARRAY['Tyler', 'Claudia', 'Brian'])

SELECT *
FROM customers
WHERE last_name LIKE ANY (ARRAY['%Mc%', '%Mac%'])